# 2 implementation

## 1:  BeautifulSoup

In [1]:
from bs4 import BeautifulSoup
import scrapy
from scrapy.crawler import CrawlerProcess
import requests
import json

In [2]:
class Mybs():
    def __init__(self, category = "dining-chairs", page_num = 1):
        self.category = category
        self.page_num = page_num
        self.urls = [f"https://www.houzz.com/products/{category}"]
        if not page_num == 1:
            for pa in range(36, int(page_num) * 36, 36):
                self.urls.append(f"https://www.houzz.com/products/{category}/p/{pa}")
        self.bsfile = open(f"bs_{category}.jl", "w", newline="")
    
    def crawl_mybs(self):
        for url in self.urls:
                html = requests.get(url)
                bsobj = BeautifulSoup(html.text, 'lxml')
                descripts = bsobj.find_all('span', class_ = "hz-product-card__description_content")
                images = bsobj.find_all('img', class_ = "hz-image hz-responsive-img")
                prices = bsobj.find_all('span', class_ = "hz-product-price hz-product-price--v2 hz-product-price--final")
                for img, desc, price in zip(images, descripts, prices):
                    name = img['alt']
                    image_scr = img['src']
                    description =(desc.text.replace("\n", "  ")).replace("\r", " ").replace("\"", " ")
                    item_price = price.text
                    item = {"name": name, "image_scr": image_scr, "description": description, "price":item_price}
                    self.bsfile.write(json.dumps(item) + "\n")
        self.bsfile.close()
        

In [3]:
mybs = Mybs(category = "sofas-and-sectionals", page_num = 3)
mybs.crawl_mybs()

## 2: scrapy spider

In [4]:
import scrapy
import json
#item class
class lItem(scrapy.Item):
    description = scrapy.Field()
    price = scrapy.Field()
    img_link = scrapy.Field()
    name = scrapy.Field()
#spider class    
class HouzzSpider(scrapy.Spider):
    name = "houzz"

    def __init__(self, categ="dining-chairs", page_num="0", *args, **kwargs):
        super(HouzzSpider, self).__init__(*args, **kwargs)
        self.start_urls = [f'https://www.houzz.com/products/{categ}']
        if not int(page_num) == 0:
            for pa in range(36, int(page_num) * 36, 36):
                self.start_urls.append(f"https://www.houzz.com/products/{categ}/p/{pa}")
        self.file = open(f"{categ}.jl", "w", newline="")

    def parse(self, response):
        item = lItem()

        descriptions = response.xpath("//span[@class = 'hz-product-card__description_content']/descendant::text()").extract()
        img_links = response.xpath("//img[@class =' hz-image hz-responsive-img']/@src").extract()
        names = response.xpath("//img[@class =' hz-image hz-responsive-img']/@alt").extract()
        prices = response.xpath("//span[@class ='hz-product-price hz-product-price--v2 "
                                "hz-product-price--final']/descendant::text()").extract()

        for name, description, img_link, price in zip(names, descriptions, img_links, prices):
            item['description'] = (description.replace("\n", "  ")).replace("\r", " ").replace("\"", " ")
            item['img_link'] = img_link
            item['name'] = name
            item['price'] = price
            self.file.write(json.dumps(dict(item)) + "\n")
            yield item

    def close(self, reason):
        self.file.close()


In [5]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess({})
process.crawl(HouzzSpider)
process.start()
##it is also able to get category as categ in inpute and number of page for crawl as page_num in inpute
##defulte categ = "dining-chairs" and page_num = 0 (crawl just first page)
##for get more page with different category try below in scrapy shell(jl file as result saved):
#############################################
##        scrapy crawl houzz -a categ=beds-and-headboards -a page_num=3
#############################################

2021-01-04 20:59:53 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-01-04 20:59:53 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-01-04 20:59:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-01-04 20:59:53 [scrapy.crawler] INFO: Overridden settings:
{}
2021-01-04 20:59:53 [scrapy.extensions.telnet] INFO: Telnet Password: a0f8cd534d53d625
2021-01-04 20:59:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-01-04 20:59:53 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.houzz.com/products/dining-chairs>
{'description': 'Lend your dining room a Hollywood-inspired vibe with this '
                'chic Heidi Black Velvet Dining Chair. This pretty chair '
                'features velvety smooth upholstery on its back and seat '
                'cushions to comfort and pamper guests while they enjoy their '
                'meals. The black velvet and gold metal combination ensures '
                'that you make a big statement in your space. The metal frame '
                'has a polished gold finish that adds to the lavish look of '
                'this armless chair.Polished Gold Metal FramePlush Black '
                'VelvetLight Assembly: Simply Attach Base',
 'img_link': 'https://st.hzcdn.com/fimgs/98c16ab80ae8ffb7_1610-w233-h233-b1-p10--.jpg',
 'name': 'Heidi Velvet Dining Chairs, Set of 2, Black',
 'price': '$310'}
2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG

2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.houzz.com/products/dining-chairs>
{'description': 'Dining chairs feature traditional wood paired with a modern '
                'design            Woven natural papercord seat adds a unique '
                'design element            Ideal for Home and Office Use - '
                'Living Rooms, Dining Rooms, Bedrooms, Nurseries, Waiting '
                'Areas, Parties, Banquets, etc.            Arrives fully '
                'assembled            Dimensions: Please see photos',
 'img_link': 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7',
 'name': 'Modern Dining Chairs Wood Armchairs, Set of 2, Natural',
 'price': '$360'}
2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.houzz.com/products/dining-chairs>
{'description': 'Leather Upholstered Steel Legs  Full Leather Chair   Fire '
                'Retardant  Made in Italy  Seat Height: 17.7

2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.houzz.com/products/dining-chairs>
{'description': 'Recline or dine in style with this charming Cafton Chair '
                'perfect for your favorite country-chic or boho living space. '
                'Beautifully made from birch wood, this traditional chair '
                'features a handsomely finished frame with a stylish cross '
                'back and delicately curved supports. Woven from rattan, its '
                'cozy seat offers both added comfort and style.      '
                'Features   Versatile X-back dining chair   Woven Rattan '
                'Seat   Solid Wood Construction   Tapered Legs   Arrives Fully '
                'Assembled      Product Dimensions:   Overall Dimensions: 18 W '
                "x 20 D x 35 H   Seat Dimensions: 18'W x 16.5''D   Seat Height "
                'from Ground: 18 H   Seat Back Height: 17 H',
 'img_link': 'data:image/gif;base64,R0lGODlhAQAB

2021-01-04 20:59:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.houzz.com/products/dining-chairs>
{'description': 'Savor every bite in comfort and style with the Nika Dining '
                'Chair. Crafted in stark contemporary fashion, this seat '
                'features clean straight lines, polished metal details and a '
                'simple silhouette that sits well in any dining room design.  '
                'Plastic glides  Foam upholstery: CA TB-117 rating  Colors: '
                'Matte black and polished silver  Materials: Naugahyde seat '
                'and stainless steel frame  Fabric composition: 65% polyester, '
                '35% cotton  Weight capacity: 264 lbs  Seat height: 17.8   '
                'Armrest height: 21.8   Seat back height: 15.3   Dimensions: W '
                '20.5  x D 23.8  x H 32.2 ',
 'img_link': 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7',
 'name': 'Nika Dining Chair, Seat: Matte

2021-01-04 20:59:55 [scrapy.core.engine] INFO: Closing spider (finished)
2021-01-04 20:59:55 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 239,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 123899,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'elapsed_time_seconds': 2.110604,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 1, 4, 17, 29, 55, 840979),
 'item_scraped_count': 36,
 'log_count/DEBUG': 37,
 'log_count/INFO': 10,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2021, 1, 4, 17, 29, 53, 730375)}
2021-01-04 20:59:55 [scrapy.core.engine] INFO: Spider closed (finished)


In [6]:
import pandas as pd

In [8]:
dfjl = pd.read_json('dining-chairs.jl', lines=True)
dfjl[:10]

,description,img_link,name,price
0,Creating a subtle rustic look for your dining ...,https://st.hzcdn.com/fimgs/6991c57d0f9cec5e_29...,Pemberly Row Wood Dining Swivel Counter Stools...,$208
1,Spread elegance and modernity to your dining r...,https://st.hzcdn.com/fimgs/1c610f4f0dd70537_58...,"Xavier Velvet Dining Chair, Gray, Gold Legs",$238
2,"Elegant and eye-catching, the stunning navy ve...",https://st.hzcdn.com/fimgs/20518daf0bac0ce1_82...,"Akoya Velvet Dining Chair, Set of 2, Navy",$353
3,Extend contemporary panache to the dining room...,https://st.hzcdn.com/fimgs/c201100b0ae8ffea_21...,"Karina Velvet Dining Chairs, Set of 2, Gray, G...",$263
4,Featuring button-tufted velvet upholstery on a...,https://st.hzcdn.com/fimgs/11d1261f0a4bd92f_71...,"Lemele Velvet Dining Chairs, Set Of 2, Brown S...",$316
5,Lend your dining area a touch of retro charm w...,https://st.hzcdn.com/fimgs/fa11f12a0ae8ffb7_95...,"Gianna Velvet Dining Chair, Gray, Gold Base",$195
6,Lend your dining room a Hollywood-inspired vib...,https://st.hzcdn.com/fimgs/98c16ab80ae8ffb7_16...,"Heidi Velvet Dining Chairs, Set of 2, Black",$310
7,Dine in luxurious style with the Nikki navy ve...,https://st.hzcdn.com/fimgs/46e12cae0db1c272_73...,"Nikki Velvet Dining Chairs, Set of 2, Navy",$326
8,Make a smooth move at your dining area with th...,https://st.hzcdn.com/fimgs/38f195490ae8fffe_95...,"Giselle Velvet Dining Chairs, Set of 2, Pink, ...",$299
9,The sienna dining chair��_s minimalist aesthet...,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...","Sienna Leather Dining Chair, Matte Ochre",$344
